In [1]:
import tensorflow as tf

### Take a note of datatypes

In [54]:
x = tf.add(1.0,2.0)

In [55]:
x

<tf.Tensor: id=29, shape=(), dtype=float32, numpy=3.0>

In [56]:
print(x)

tf.Tensor(3.0, shape=(), dtype=float32)


In [57]:
type(x.numpy())

numpy.float32

In [22]:
x1 = tf.constant(1,shape=(5,5),dtype=tf.float64)

In [23]:
x1

<tf.Tensor: id=20, shape=(5, 5), dtype=float64, numpy=
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])>

In [25]:
print(x1)

tf.Tensor(
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]], shape=(5, 5), dtype=float64)


In [26]:
x2 = tf.constant(1.0,dtype=tf.float64)

In [28]:
print(tf.add(2.0,x2))

tf.Tensor(3.0, shape=(), dtype=float64)


### Numpy compatability

In [45]:
import numpy as np

In [47]:
ndarray = np.ones([3, 3])
ndarray.dtype

dtype('float64')

In [48]:
ndarray = np.ones([3, 3],dtype='float32')
ndarray.dtype

dtype('float32')

In [49]:
print(tf.multiply(ndarray,42))

tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float32)


### List Devices

In [52]:
print(tf.config.experimental.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]


XLA_CPU : Its the XLA Compiler exposed as a device

### tf.data API

#### tensor_slices

> Use when each data point is held in a separate file

In [87]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

In [83]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

Elements of ds_tensors:
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)


In [84]:
train_size = int(0.8 * 6)
train_dataset = ds_tensors.take(train_size)
test_dataset = ds_tensors.skip(train_size)

In [85]:
for x in train_dataset:
  print(x)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


In [86]:
for x in test_dataset:
  print(x)

tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)


In [5]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

In [6]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

Elements of ds_tensors:
tf.Tensor([1 9], shape=(2,), dtype=int32)
tf.Tensor([ 4 25], shape=(2,), dtype=int32)
tf.Tensor([36 16], shape=(2,), dtype=int32)


#### Iterator

In [94]:
it = iter(ds_tensors)

In [95]:
next(it)

<tf.Tensor: id=40585, shape=(), dtype=int32, numpy=1>

In [96]:
next(it)

<tf.Tensor: id=40586, shape=(), dtype=int32, numpy=2>

> **Shows OutOfRange exception with end is reached**

#### TextLineDataset

> When each line of a file is a data point

In [21]:
filename = "data/gemm2000.txt"
ds_file = tf.data.TextLineDataset(filename)

In [23]:
print('\nElements in ds_file:')
for x in ds_file:
  print(x)
  break


Elements in ds_file:
tf.Tensor(b'10\t0.0001243\t2000.0', shape=(), dtype=string)


In [25]:
ds_file

<TextLineDatasetV2 shapes: (), types: tf.string>

In [26]:
it = iter(ds_file)

In [29]:
x = next(it).numpy()
x

b'10\t1.29e-5\t2000.0'

In [43]:
t = tf.constant(float(x.split()[1]), dtype="float64")

In [44]:
t

<tf.Tensor: id=40146, shape=(), dtype=float64, numpy=1.29e-05>

In [45]:
y = next(it)

In [46]:
y

<tf.Tensor: id=40147, shape=(), dtype=string, numpy=b'10\t1.23e-5\t2000.0'>

In [52]:
tf.strings.split(y)

<tf.Tensor: id=40205, shape=(3,), dtype=string, numpy=array([b'10', b'1.23e-5', b'2000.0'], dtype=object)>

In [59]:
ds_file = tf.data.TextLineDataset(filename)
ds_file = ds_file.map(tf.strings.split)

In [61]:
y = next(iter(ds_file))

In [73]:
y

<tf.Tensor: id=40460, shape=(3,), dtype=string, numpy=array([b'10', b'0.0001243', b'2000.0'], dtype=object)>

In [68]:
for i in y:
    print(tf.strings.to_number(i))

tf.Tensor(10.0, shape=(), dtype=float32)
tf.Tensor(0.0001243, shape=(), dtype=float32)
tf.Tensor(2000.0, shape=(), dtype=float32)


In [76]:
def cast_to_number(x):
    return tf.strings.to_number(x,out_type=tf.float64)

In [75]:
tf.strings.to_number(y,out_type=tf.float64)

<tf.Tensor: id=40509, shape=(3,), dtype=float64, numpy=array([1.000e+01, 1.243e-04, 2.000e+03])>

In [77]:
ds_file = ds_file.map(cast_to_number)

In [78]:
y = next(iter(ds_file))
y

<tf.Tensor: id=40521, shape=(3,), dtype=float64, numpy=array([1.000e+01, 1.243e-04, 2.000e+03])>

In [2]:
x = [1,2]

In [4]:
t = tf.constant(x)

In [5]:
t

<tf.Tensor: id=1, shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>

In [7]:
y = tf.constant(3)

In [10]:
y2 = y**2

In [11]:
y,y2

(<tf.Tensor: id=2, shape=(), dtype=int32, numpy=3>,
 <tf.Tensor: id=6, shape=(), dtype=int32, numpy=9>)

In [18]:
tf.stack([y,y2])[...,tf.newaxis]

<tf.Tensor: id=14, shape=(2, 1), dtype=int32, numpy=
array([[3],
       [9]], dtype=int32)>